# GDrive Downloader 

### Mount Google Drive
To stream files we need to mount Google Drive.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

## Install requirements

####For DDL

In [0]:
!sudo apt-get install wget

####For Torrent

In [0]:
!apt install python3-libtorrent

####For Mega

In [0]:
!sudo apt-get install megatools

## Start Download

####DDL

In [0]:
import os
import subprocess
import re

link = input("Paste the DDL Link: ")
savepath = "/content/drive/My Drive/Downloads"

name = os.path.basename(link)
file = name.split("?", maxsplit=1)[0]
filename = re.sub('%20', ' ', file)

subprocess.call("wget '{}' -O '{}/{}'".format(link, savepath, filename), shell=True)

###MEGA (Mega gives 5GB per 8hour)

In [0]:
!megadl --path '/content/drive/My Drive/Downloads/Mega/' --proxy '' - ''

In [0]:
import os
import subprocess
import requests
import traceback
import time
from lxml.html import fromstring
from itertools import cycle

def get_proxies():
  
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies


user = input("Username: ")

savepath = "/content/drive/My Drive/Downloads/Mega/New/"

i = 0

def reload():
  proxies = get_proxies()
  pool = cycle(proxies)
  print(proxies)
  proxy = next(pool)
  
  try:
    result = subprocess.call("megadl --path '{}' --proxy '{}' - '{}'".format(savepath, proxy, link), shell=True)
    print(result)
    
    if result is 1 :
      print("Trying proxies again in 10 seconds")
      time.sleep(10)
      reload()
    
    else :
      print("Done")
  except:
    print("Error")
    
    
if user :
  
  password= input("password: ")
  remote = input("foldername: ")
  subprocess.call("megaget -u '{}' -p '{} --path '{} '/Root/{}'".format(user, password, savepath, remote), shell=True)

else:
  link = input("Paste the Mega Public Link: ")
  
  try:
    result = subprocess.call("megadl --path '{}' - '{}'".format(savepath, link), shell=True)
    print(result)
    
    if result is 1 :
      print("Trying Proxies...")
      reload()
    else :
      print("Done")
  except:
    print("Error")

###Torrent

Load Torrent File (if you have magnet link then skip)

In [0]:
from google.colab import files
uploaded = files.upload()

Torrent Downlad

In [0]:
import libtorrent as lt
import time
import sys

ses = lt.session()
ses.listen_on(6881, 6891)

try:
  info = lt.torrent_info(list(uploaded.keys())[0])
  h = ses.add_torrent({"ti": info, "save_path": "/content/drive/My Drive/Torrent"})
  print("starting", h.name())

except:
  link = input("Paste the magnet link: ") 
  h = lt.add_magnet_uri(ses, link, {"save_path": "/content/drive/My Drive/Torrent"})
  while (not h.has_metadata()):
    time.sleep(.1)
  info = h.get_torrent_info()
  print("starting", h.name())

i = 0
k = 0
p = 0

ratio = input("Ratio(press enter without input to stop torrent after downloading): ")

if ratio:
  ratio = float(ratio)
  print("Torrent will be stopped after acheiving above ratio.")
else:
  ratio = 100
  timer = input("Timer(press enter without input to stop torrent after downloading): ")
  if timer:
    timer = int(timer)
  else:
    timer = 60
  j = int(timer * 6)

while i <= j and k < ratio and p != 1:
    
    i += 1   
    
    s = h.status()
    
    state_str = [
        "queued",
        "checking",
        "downloading metadata",
        "downloading",
        "finished",
        "seeding",
        "allocating",
        "checking fastresume",
    ]
      
    print(
        "\r%.2f%% done (lc: %.2f MB | sd: %.2f MB | dn: %.1f MB/s | up: %.1f MB/s | prs: %d | rto: %.2f) %s"
        % (
            s.progress * 100,
            s.total_download / 1048576,
            s.total_upload / 1048576,
            s.download_rate / 1048576,
            s.upload_rate / 1048576,
            s.num_peers,
            s.total_upload / s.total_download,
            state_str[s.state],
        )
    )
    if state_str[s.state] is "seeding":
      print("Now Seeding...")
      if s.total_download is 0 :
        k = 0
      else:
        k = int(s.total_upload / s.total_download)
      
      if ratio is 100 and timer is 60:
        p = 1
      else:
        p = 1
    else:
      p = 0
    
    time.sleep(10)

print(h.name(), "complete")
print (i, j, k, ratio, p)
rem = input("Do you want to remove the torrent from list(Does not delete anything in google drive)?(y/n): ")
if rem is 'y':
  ses.remove_torrent(h)
  print("Torrent deleted")
else:
  print("Torrent not deleted.")